# Python Multithreading


Copyright (c) 2016-2017 Duc Nguyen. Released under GPLv3.

Note:
- *This script is written for Python 3.5. Other Python verions may not work as expected.*
- *Each code block is independent from each other, to run an example, just run a code block, you don't need to run from top to bottom*

## Create new thread

New thread can easily be created by initiating a new `threading.Thread` instance. There are two ways to initiate and run a thread:

- Define a function and assign that function to parameter `target` of the new thread, then run the Thread by calling its `start()` method. If the assigned function takes any arguments, include those arguments inside a tuple and assign that tuple to `args`
- Subclass `Thread.run()`, initiate that subclass and then run that subclass with `.start()`. Calling `.start()` will execute the thread's `run()`

In [4]:
import threading
import time

def worker(thread_name, counter):
    """ Print the thread name every 1 second for *counter* times """
    for _idx in range(counter):
        time.sleep(1)
        print("{} - {}".format(thread_name, _idx))

class DummyThread(threading.Thread):
    def __init__(self, name, counter):
        super(DummyThread, self).__init__()
        self.name = name
        self.counter = counter
    def run(self):
        for _idx in range(self.counter):
            time.sleep(1)
            print("{} (initialized from DummyThread) - {}".format(self.name, _idx))
            
# Create thread0 by assigning function to `target`, and assign the parameters to function through `args`
thread0 = threading.Thread(target=worker, args=("Thread 0", 2))
thread0.start()

# Create thread1 by initialize DummyThread
thread1 = DummyThread("Thread 1", 5)
thread1.start()

Thread 1 (initialized from DummyTrhead) - 0
Thread 0 - 0
Thread 1 (initialized from DummyTrhead) - 1
Thread 0 - 1
Thread 1 (initialized from DummyTrhead) - 2
Thread 1 (initialized from DummyTrhead) - 3
Thread 1 (initialized from DummyTrhead) - 4


### Threads can also be created and started from other working threads

In the below example, a thread named "Inside thread" is created from "Thread 0". Notice that even though "Inside thread" is created inside "Thread 0", "Thread 0" can actually terminates before "Inside thread". An intuition can be obtained from this is that each thread can be considered as an independent code execution.

In [5]:
import threading
import time

def worker(thread_name, counter):
    """ Print the thread name every 1 second for *counter* times """
    for _idx in range(counter):
        time.sleep(1)
        print("{} - {}".format(thread_name, _idx))
    print("{} is exitting...".format(thread_name))

class DummyThread(threading.Thread):
    
    def __init__(self, name, counter):
        super(DummyThread, self).__init__()
        self.name = name

    def run(self):
        print("{} is running".format(self.name))
        print("Creating a new thread from {}...".format(self.name))
        thread1 = threading.Thread(target=worker, args=("Inside thread", 5))
        thread1.start()
        print("{} is exitting...".format(self.name))
            
# Create thread0 by initialize DummyThread
thread0 = DummyThread("Thread 0", 5)
thread0.start()

Thread 0 is running
Creating a new thread from Thread 0...
Thread 0 is exitting...
Inside thread - 0
Inside thread - 1
Inside thread - 2
Inside thread - 3
Inside thread - 4
Inside thread is exitting...


## Data sharing between threads

Spawned threads can access data from thread that defines them. As a result, to share data between threads, just create that mutual data inside the scope that defines all involving threads (e.g the main thread)

#### Example 1:

In [15]:
import threading

arbitrary_data = 1

def worker(thread_name):
    """ Print the thread name and the value of global's arbitrary_data """
    print("{} - arbitrary_data: {} (worker thread can access global variables)".format(thread_name, arbitrary_data))

    
thread0 = threading.Thread(target=worker, args=("Thread 0",))
thread0.start()

# Increment arbitrary_data from global scope
arbitrary_data += 1
thread1 = threading.Thread(target=worker, args=("Thread 1",))
thread1.start()

Thread 0 - arbitrary_data: 1 (worker thread can access global variables)
Thread 1 - arbitrary_data: 2 (worker thread can access global variables)


#### Example 2:

The DummyThread0_child is defined inside global scope (since `worker()` is defined as a global function). As a result, DummyThread0_child accesses `arbitrary_data` from global scope, rather than from `DummyThread0` scope.

The DummyThread1_child is defined inside `DummyThread1` scope (since `worker()` is defined as a local function). As a result, DummyThread1_child will first try to access `arbitrary_data` from its local scope.

The DummyThread2_child is defined inside `DummyThread2` scope. As a result, DummyThread2_child will first try to access `arbitrary_data` from its local scope. However, since `DummyThread2.run` does not have `arbitrary_data` in its local namespace, DummyThread2_child then tries to find `arbitrary_data` in the parent scope of `DummyThread2`.

In [23]:
import threading

arbitrary_data = "data from main thread"

def worker(thread_name):
    """ Print the thread name and the value of arbitrary_data """
    print("{} - Arbitrary data: {}".format(thread_name, arbitrary_data))
    
class DummyThread0(threading.Thread):
    
    def __init__(self, name):
        super(DummyThread0, self).__init__()
        self.name = name

    def run(self):
        arbitrary_data = "data from non-main thread" 
        dummythread0_child = threading.Thread(target=worker, args=("DummyThread0_child",))
        dummythread0_child.start()
    
    
class DummyThread1(threading.Thread):
    
    def __init__(self, name):
        super(DummyThread1, self).__init__()
        self.name = name

    def run(self):
        arbitrary_data = "data from non-main thread"
        
        # worker definition is the only difference between DummyThread0 and DummyThread1
        def worker(thread_name):
            """ Print the thread name and the value of arbitrary_data """
            print("{} - Arbitrary data: {}".format(thread_name, arbitrary_data))
        
        dummythread1_child = threading.Thread(target=worker, args=("DummyThread1_child",))
        dummythread1_child.start()
        
class DummyThread2(threading.Thread):
    
    def __init__(self, name):
        super(DummyThread2, self).__init__()
        self.name = name

    def run(self):        
        
        # does not have local arbitrary_data
        
        def worker(thread_name):
            """ Print the thread name and the value of arbitrary_data """
            print("{} - Arbitrary data: {}".format(thread_name, arbitrary_data))
        
        dummythread2_child = threading.Thread(target=worker, args=("DummyThread2_child",))
        dummythread2_child.start()
        

thread0 = DummyThread0("Thread 0")
thread0.start()

thread1 = DummyThread1("Thread 1")
thread1.start()

thread2 = DummyThread2("Thread 2")
thread2.start()

DummyThread0_child - Arbitrary data: data from main thread
DummyThread1_child - Arbitrary data: data from non-main thread
DummyThread2_child - Arbitrary data: data from main thread


### Shared primitive data cannot be modified by worker threads

Worker threads cannot modify primitive data from the parent scope (just like function). Attempt to do so will create a thread-local variable, or will raise an UnboundLocalError.

In [24]:
import threading

arbitrary_data = 1

def worker_increment(thread_name):
    """ Modify arbitrary data """
    try:
        arbitrary_data += 1
        print("{} - Arbitrary data: {}".format(thread_name, arbitrary_data))
    except UnboundLocalError:
        print("UnboundLocalError was raised")
        
def worker(thread_name):
    """ Set the arbitrary data """
    arbitrary_data = 10
    print("{} - Set arbitrary data to {}".format(thread_name, arbitrary_data))
        
thread0 = threading.Thread(target=worker_increment, args=("Thread increment",))
thread0.start()

thread1 = threading.Thread(target=worker, args=("Thread set",))
thread1.start()

# Temporarily block the main thread until both thread0 and thread1 terminate
thread0.join()
thread1.join()

print("arbitrary_data from main thread: {}".format(arbitrary_data))

UnboundLocalError was raised
Thread set - Set arbitrary data to 10
arbitrary_data from main thread: 1


### Shared objects can be modified by worker threads

Worker threads can access and modify object data from parent scope (just like function).

In [28]:
import threading

arbitrary_data = []

def worker_append(thread_name, value):
    """ Append `value` to `arbitrary_data` """
    arbitrary_data.append(value)
    
def worker_len():
    """ Check the length of `arbitrary_data """
    print("(Worker thread) Length arbitrary_data: {}".format(len(arbitrary_data)))

print("Length arbitrary_data before: {}".format(len(arbitrary_data)))
thread0 = threading.Thread(target=worker_append, args=("Thread append", 1))
thread0.start()

# Temporarily block main thread until thread0 terminates
thread0.join()
print("Length arbitrary_data after: {}".format(len(arbitrary_data)))

thread1 = threading.Thread(target=worker_len)
thread1.start()

Len arbitrary_data before: 0
Len arbitrary_data after: 1
(Worker thread) Length arbitrary_data: 1


## Thread synchronization

## Other helper functions

### Determine the current threads

In [ ]:
import threading
import time

def worker(thread_name):
    """ Print the thread name and the value of global's arbitrary_data """
    print("{} - arbitrary_data: {}".format(thread_name, arbitrary_data))

def worker_set(thread_name):
    """ Print the thread name and change the value of global's primitive arbitrary_data """
    try:
        arbitrary_data += 10
        print("{} - arbitrary_data: {}".format(thread_name, arbitrary_data))
    except UnboundLocalError:
        print("arbitrary_data does not have local scope, {} fails".format(thread_name))